# Modelos Discriminativos - Parte 2 - Sistemas Não-Lineares

Os sistemas lineares têm a vantagem de serem facilmente treinados através de um processo de otimização convexa, isto é, que só tem um ponto de mínimo. Apesar disso, 

## Objetivos

Ao fim desta iteração, o aluno será capaz de:
* Entender o conceito de *aproximação universal*
* Entender os algoritmos de *backpropagation*
* Aplicar redes MLP para problemas de classificação
* Configurar redes MLP quanto ao seu número de neurônios e número de camadas

In [2]:
# Inicializacao
%matplotlib inline

import numpy as np
from matplotlib import pyplot as plt

# Abrindo conjunto de dados
import csv
with open("biometria.csv", 'rb') as f:
    dados = list(csv.reader(f))
    
rotulos_volei = [d[0] for d in dados[1:-1] if d[0] is 'V']
rotulos_futebol = [d[0] for d in dados[1:-1] if d[0] is 'F']
altura_volei = [[float(d[1])] for d in dados[1:-1] if d[0] is 'V']
altura_futebol = [[float(d[1])] for d in dados[1:-1] if d[0] is 'F']
peso_volei = [[float(d[2])] for d in dados[1:-1] if d[0] is 'V']
peso_futebol = [[float(d[2])] for d in dados[1:-1] if d[0] is 'F']

## Sistemas não-lineares

Um sistema não-linear é aquele que não obedece as condições de linearidade. De forma geral, ele pode ser escrito como um vetor de saídas $\boldsymbol y$ que é definido em função de um vetor de entradas $\boldsymbol x$:





$$\boldsymbol y = f(\boldsymbol A \boldsymbol x),$$

onde $\boldsymbol y$ e $\boldsymbol x$ são dados do problema e $\boldsymbol A$ é uma matriz que combina linearmente os elementos das entradas. Uma possível função $f(.)$ é a tangente hiperbólica, que resulta num sistema:

In [10]:
def nl_forward(A, x):
    return np.tanh(np.dot(A,x))

A = np.array([[-1, 0], [0, 1], [0.5, 0.5]]) # Entram dois elementos e saem três
x = np.array([[-0.5, 0, 0.1, 0.3], [0.7, 0.1, 0, 10]]) # Entra Quatro vetores-coluna

print nl_forward(A, x)

[[ 0.46211716  0.         -0.09966799 -0.29131261]
 [ 0.60436778  0.09966799  0.          1.        ]
 [ 0.09966799  0.04995837  0.04995837  0.99993274]]


Porém, lembremos: queremos que nossa função tenha um comportamento específico, e não um comportamento arbitrário e/ou aparentemente aleatório. Idealmente, gostaríamos que os pesos fossem ajustados de forma a reduzir o erro das saídas em relação a um conjunto de dados de treinamento. À partir disso, verificaremos como o comportamento de nossa função é generalizado, avaliando seu comportamento em dados de teste.

O erro de aproximação, na etapa de treinamento, é dado por:

$$ E = ||\boldsymbol y_e - \boldsymbol y||^2 = \boldsymbol y_e^2 - 2 \boldsymbol y_e \boldsymbol y + \boldsymbol y^2$$

Logo, temos:
$$ \frac{dE}{d \boldsymbol y_e} = 2\boldsymbol y_e - 2 \boldsymbol y$$

Podemos usar a regra da cadeia para evidenciar uma expressão para a derivada do erro em relação aos coeficientes da matriz $\boldsymbol A$:

$$\frac{dE}{d \boldsymbol A} = \frac{dE}{d \boldsymbol y_e} \frac{d \boldsymbol y_e}{d\boldsymbol A} = \frac{dE}{d \boldsymbol y_e} \frac{d \boldsymbol f(\boldsymbol A \boldsymbol x)}{d\boldsymbol A \boldsymbol x} \frac{d \boldsymbol A \boldsymbol x}{d \boldsymbol A}$$

Se cada vetor de dados é representado como um vetor-coluna, então:
 $$\frac{d \boldsymbol A \boldsymbol x}{d \boldsymbol A} = x^T$$
 
e, portanto:

$$\frac{dE}{d \boldsymbol A} = 2 (y_e - y) (f'(\boldsymbol A \boldsymbol x) \otimes f'(x^T)),$$

onde $\otimes$ denota a multiplicação elemento-a-elemento.

A derivada do erro em relação a $ \boldsymbol A$, chamado também de gradiente, indica a direção na qual o erro mais aumenta. Portanto, seu oposto indica a direção em que $\boldsymbol A$ pode se mover de forma a reduzir o erro ao máximo. Asso, podemos definir nossa função não-linear na forma: